In [48]:
!pip install pretrainedmodels
!pip install imutils
!pip install opencv-python

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import joblib
import cv2
import os
import time
import random
import pretrainedmodels
import numpy as np

from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load torch...!!!
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Load torchvision ...!!!
from torchvision import transforms

'''SEED Everything'''
def seed_everything(SEED=42):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True # keep True if all the input have same size.
SEED=42
seed_everything(SEED=SEED)
'''SEED Everything'''

'SEED Everything'

In [71]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # GPU
epochs = 5 # Number of epochs
BS = 16 # Batch size

image_paths = list(paths.list_images('./101_ObjectCategories'))


data = []
labels = []
unique_labels = {}
for img_path in tqdm(image_paths):
    label = img_path.split(os.path.sep)[-2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
    
    if label in labels and (unique_labels[label] < 30):
        unique_labels[label] += 1
        data.append(resized)
    elif label not in unique_labels:
        unique_labels[label] = 1
        data.append(resized)
    else:
        continue
        
    labels.append(label)

data = np.array(data)
labels = np.array(labels)
print(data.shape)
print(labels.shape)

100%|██████████████████████████████████████| 9144/9144 [00:23<00:00, 383.87it/s]


(3060, 224, 224)
(3060,)


In [72]:
lb = LabelEncoder()
labels = lb.fit_transform(labels)
print(f"Total Number of Classes: {len(lb.classes_)}")

Total Number of Classes: 102


In [73]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])   

In [74]:
# divide the data into train, validation, and test set
(X, x_val , Y, y_val) = train_test_split(data, labels, test_size=0.2,  stratify=labels,random_state=42)
(x_train, x_test, y_train, y_test) = train_test_split(X, Y, test_size=0.25, random_state=42)
print(f"x_train examples: {x_train.shape}\nx_test examples: {x_test.shape}\nx_val examples: {x_val.shape}")

x_train examples: (1836, 224, 224)
x_test examples: (612, 224, 224)
x_val examples: (612, 224, 224)


In [81]:
from scipy import io

data = {
    'labelset': x_train.reshape(1836, 50176),
    'unlabelset': x_val.reshape(612, 50176),
    'testset': x_test.reshape(612, 50176)
}
io.savemat('./caltech_101.mat', data)